In [1]:
from pandas import read_csv
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
import xgboost as xgb
import matplotlib.pyplot as plt
import numpy as np
import data
import lightgbm as lgb
from sklearn.metrics import log_loss # positive value

In [2]:
dataset = data.Dataset()

#900
# cols=['v275b_N2_UA12', 'v231b_VE', 'v231b_BG', 'v231b_FI', 'v275b_N2_ITH2', 'v251b_GB', 'v275b_N2_NL13', 'v251b_ME', 'v231b_AO', 'v228b_MD', 'v275b_N2_PL81', 'v228b_HU', 'v228b_145', 'v231b_IS', 'v233b_LT', 'v228b_AM', 'v275b_N2_AZ-XAC', 'v228b_IQ', 'v228b_SE', 'v228b_NO', 'v251b_30', 'v231b_AZ', 'v275b_N2_AZ-SM', 'v275b_N2_UKG1', 'v275b_N2_UKD6', 'v228b_MK', 'v233b_BG', 'v233b_SI', 'v275b_N2_AZ-BIL', 'v275b_N2_PL82', 'v275b_N2_AZ-SAB', 'v275b_N2_UKD4', 'v233b_FI', 'v228b_US', 'v275b_N2_UKL2', 'v231b_EE', 'v275b_N2_AZ-QAB', 'v231b_CSXX', 'v251b_RO', 'v275b_N2_FRE2', 'v275b_N2_ITF5', 'v228b_DZ', 'v231b_GB', 'v275b_N2_PL42', 'v275b_N2_PT15', 'v275b_N2_ES21', 'v275b_N2_UA22', 'v275b_N2_UKE4', 'v228b_LK', 'v231b_-1', 'v275b_N2_ES53', 'v251b_BE', 'v275b_N2_UKE3', 'v275b_N2_UKC1', 'v228b_-1', 'v251b_MK', 'v251b_143', 'v228b_AT', 'v233b_US', 'v275b_N2_UKI4', 'v275b_N2_ITH1', 'v228b_GP', 'v231b_145', 'v275b_N2_UKC2', 'v275b_N2_UA23', 'v228b_RS', 'v275b_N2_ITI2', 'v233b_AM', 'v233b_NO', 'v275b_N2_UKJ3', 'v228b_NL', 'v251b_SK', 'v228b_FI', 'v281a_ca', 'v275b_N2_UA52', 'v251b_AZ', 'v275b_N2_UA82', 'v251b_HR', 'v275b_N2_BABRC', 'v231b_CZ', 'v275b_N2_AZ-BAR', 'v228b_PT', 'v231b_SE', 'v275b_N2_NL12', 'v275b_N2_UKM8', 'v275b_N2_UKE1', 'v233b_IE', 'v231b_IE', 'v275b_N2_FRF3', 'v275b_N2_ITF2', 'v228b_15', 'v275b_N2_UA24', 'v275b_N2_UKD7', 'v228b_ES', 'v231b_AM', 'v228b_LT', 'v228b_MA', 'v228b_XK', 'v228b_HR', 'v233b_SE', 'v275b_N2_AZ-SAL', 'v275b_N2_AT11', 'v275b_N2_AT34', 'v233b_NL', 'v251b_FR', 'v231b_LT', 'v231b_LK', 'v233b_CZ', 'v233b_MA', 'v275b_N2_UKL1', 'v231b_RO', 'v275b_N2_ES62', 'v275b_N2_UA43', 'v251b_SI', 'v275b_N2_ITI3', 'v275b_N2_ITH4', 'v231b_143', 'v275b_N2_UKM7', 'v251b_5', 'v275b_N2_FRF1', 'v251b_34', 'v231b_FR', 'v275b_N2_ITF6', 'v231b_DZ', 'v275b_N2_ITH3', 'v275b_N2_UKG3', 'v275b_N2_AZ-ABS', 'v275b_N2_AZ-BAL', 'v275b_N2_FRB0', 'v231b_CSHH', 'v228b_11', 'v275b_N2_NL23', 'v233b_HU', 'v233b_GE', 'v233b_BY', 'v275b_N2_UA14', 'v275b_N2_NL42', 'v275b_N2_NO02', 'v231b_LV', 'v251b_XK', 'v233b_LK', 'v275b_N2_ITG2', 'v275b_N2_FRJ1', 'v275b_N2_UA81', 'v251b_ES', 'v231b_14', 'v275b_N2_UA84', 'v275b_N2_AZ-AST', 'v231b_15', 'v233b_11', 'v231b_GE', 'v275b_N2_UKK1', 'v228b_SK', 'v275b_N2_AZ-SKR', 'v275b_N2_UA13', 'v233b_15', 'v275b_N2_NO04', 'v231b_BE', 'v231b_11', 'v275b_N2_RU81', 'v231b_US', 'v228b_14', 'v275b_N2_NL31', 'v275b_N2_ES12', 'v231b_HU', 'v275b_N2_UA51', 'v233b_MK', 'v275b_N2_AZ-GYG', 'v251b_IT', 'v231b_GR', 'v275b_N2_UKH1', 'v275b_N2_ES41', 'v251b_35', 'v231b_PT', 'v231b_AT', 'v275b_N2_PL63', 'v275b_N2_AZ-GAD', 'v251b_PL', 'v275b_N2_UKJ2', 'v275b_N2_UKJ1', 'v233b_BE', 'v275b_N2_PL84', 'v233b_HR', 'v228b_TR', 'v233b_GR', 'v275b_N2_PT18', 'v275b_N2_AZ-HAC', 'v233b_143', 'v233b_AT', 'v251b_PT', 'v275b_N2_ES42', 'v275b_N2_UA11', 'v251b_BA', 'v275b_N2_FRJ2', 'v228b_RO', 'v233b_DZ', 'v275b_N2_HU32', 'v251b_UA', 'v275b_N2_PL51', 'v228b_143', 'v275b_N2_UKD3', 'v233b_SK', 'v275b_N2_ES11', 'v275b_N2_NO06', 'v275b_N2_ES52', 'v275b_N2_UKI7', 'v275b_N2_ES70', 'v233b_ES', 'v233b_14', 'v233b_XK', 'v275b_N2_FRG0', 'v231b_ES', 'v233b_FR', 'v275b_N2_UKG2', 'v228b_FR', 'v231b_BY', 'v233b_TR', 'v275b_N2_NL11', 'v275b_N2_PL61', 'v228b_PL', 'v233b_RO', 'v275b_N2_FRL0', 'v275b_N2_UA62', 'v275b_N2_UA83', 'v275b_N2_UA53', 'v231b_MA', 'v275b_N2_PL41', 'v275b_N2_UKM9', 'v275b_N2_PL92', 'v233b_PT', 'v275b_N2_ITC3', 'v231b_HR', 'v228b_34', 'v231b_SK', 'v275b_N2_PL71', 'v275b_N2_RU51', 'v228b_UA', 'v275b_N2_PL21', 'v275b_N2_AZ-ZAR', 'v275b_N2_AZ-MAS', 'v275b_N2_FRD1', 'v275b_N2_FRH0', 'v228b_35', 'v275b_N2_UA21', 'v231b_XK', 'v275b_N2_PL22', 'v275b_N2_UA42', 'v275b_N2_HU22', 'v233b_RS', 'v275b_N2_RU71', 'v275b_N2_CZ01', 'v228b_IT', 'v275b_N2_AZ-QAZ', 'v275b_N2_RO11', 'v275b_N2_NO07', 'v275b_N2_UA63', 'v275b_N2_AZ-AGU', 'v233b_35', 'v275b_N2_BY07', 'v275b_N2_AZ-SAT', 'v231b_RS', 'v275b_N2_ES51', 'v275b_N2_UA31', 'v251b_DE', 'v275b_N2_RO12', 'v275b_N2_FRC1', 'v275b_N2_PL91', 'v275b_N2_UA41', 'v231b_TR', 'v275b_N2_HU31', 'v275b_N2_AZ-GA', 'v233b_5', 'v275b_N2_ITF3', 'v275b_N2_AZ-MI', 'v275b_N2_CZ02', 'v231b_PL', 'v275b_N2_ITH5', 'v275b_N2_RO32', 'v275b_N2_AZ-TOV', 'v275b_N2_NL21', 'v275b_N2_CH07', 'v231b_5', 'v275b_N2_BG31', 'v275b_N2_ES61', 'v275b_N2_AZ-SMX', 'v275b_N2_ITI1', 'v275b_N2_AT32', 'v251b_-1', 'v275b_N2_RO41', 'v233b_IT', 'v231b_34', 'v275b_N2_ITI4', 'v228b_BA', 'v275b_N2_UKF1', 'v275b_N2_FRE1', 'v275b_N2_HU21', 'v275b_N2_RO31', 'v275b_N2_ITC1', 'v251b_RU', 'v275b_N2_UA72', 'v275b_N2_AT21', 'v275b_N2_NO03', 'v233b_UA', 'v275b_N2_RO22', 'v275b_N2_BG32', 'v275b_N2_AT33', 'v275b_N2_CZ08', 'v275b_N2_FRK2', 'v275b_N2_FI1D', 'v275b_N2_CZ07', 'v233b_PL', 'v275b_N2_NL22', 'v275b_N2_HU23', 'v275b_N2_NO05', 'v228b_5', 'v275b_N2_FRI1', 'v275b_N2_UA71', 'v275b_N2_RU21', 'v275b_N2_RO42', 'v233b_34', 'v275b_N2_FI1B', 'v275b_N2_ITF4', 'v275b_N2_DK05', 'v231b_35', 'v275b_N2_FI19', 'v275b_N2_BY04', 'v275b_N2_HU33', 'v275b_N2_FI1C', 'v275b_N2_BY06', 'v231b_IT', 'v275b_N2_HU12', 'v275b_N2_BY02', 'v275b_N2_AM03', 'v275b_N2_BY01', 'v275b_N2_ITG1', 'v275b_N2_HU11', 'v275b_N2_NL41', 'v275b_N2_AZ-QUS', 'v275b_N2_AT31', 'v275b_N2_BY03', 'v275b_N2_PT16', 'v275b_N2_NL32', 'v275b_N2_BG42', 'v231b_DE', 'v275b_N2_CH03', 'v231b_UA', 'v275b_N2_AM04', 'v24a_IT', 'v275b_N2_DK02', 'v275b_N2_CZ06', 'v24b_IT', 'v231b_BA', 'v275b_N2_ES30', 'v275b_N2_SI04', 'v275b_N2_RO21', 'v275b_N2_RU31', 'v275b_N2_PT17', 'v275b_N2_AT22', 'v233b_BA', 'v275b_N2_RU61', 'v228b_DE', 'v275b_N2_CH06', 'v233b_DE', 'v275b_N2_ITC4', 'v275b_N2_AL01', 'v275b_N2_CZ05', 'v275b_N2_NO01', 'v275b_N2_NL33', 'v275b_N2_AZ-ISM', 'v275b_N2_AT12', 'v275b_N2_UA61', 'v221_DK', 'v275b_N2_BG34', 'v275b_N2_PT11', 'v275b_N2_RS22', 'v275b_N2_SK01', 'v275b_N2_BG33', 'v275b_N2_AZ-GOR', 'v275b_N2_AM02', 'v224_DK', 'v228b_RU', 'v223_DK', 'c_abrv_NO', 'v275b_N2_RS11', 'v275b_N2_CZ04', 'v275b_N2_CH04', 'v275b_N2_AT13', 'v183_DK', 'v275b_N2_FR10', 'v275b_N2_RU11', 'v275b_N2_SK04', 'v275b_N2_CZ03', 'v275b_N2_CH01', 'v275b_N2_SK03', 'v222_DK', 'v275b_N2_BY05', 'v275b_N2_CH05', 'v275b_N2_CH02', 'v275b_N2_RU41', 'v281a_no', 'f110', 'v275b_N2_AM01', 'v275b_N2_GE01', 'v275b_N2_HR03', 'v275b_N2_DK03', 'v275b_N2_SI03', 'v275b_N2_AZ-BA', 'v275b_N2_SK02', 'f108', 'v275b_N2_BG41', 'v275b_N2_DK04', 'v281a_pt', 'c_abrv_GB', 'c_abrv_PT', 'v275b_N2_BASRP', 'v177_DK', 'v275b_N2_LT01', 'v233b_RU', 'v275b_N2_AL02', 'v275b_N2_RS21', 'v281a_et', 'v275b_N2_DK01', 'v281a_fi', 'c_abrv_FI', 'c_abrv_SI', 'v182_DK', 'v281a_mk', 'v281a_en', 'v231b_RU', 'v275b_N2_AL03', 'v281a_lv', 'v176_DK', 'v281a_sl', 'v275b_N2_IS00', 'v275b_N2_RS12', 'v281a_uk', 'c_abrv_DE', 'c_abrv_EE', 'c_abrv_IS', 'v281a_is', 'v275b_N2_EE00', 'v252_cs_GB2', 'v275b_N2_HR04', 'v281a_bs', 'v180_DK', 'v281a_es', 'v275b_N2_-4', 'c_abrv_NL', 'v243_cs_GB1', 'v275b_N2_LV00', 'c_abrv_AT', 'v262_cs_GB1', 'c_abrv_ES', 'c_abrv_LV', 'v252_cs_GB1', 'v275b_N2_BABIH', 'v275b_N2_MK00', 'v281a_pl', 'v263_cs_DE2', 'v281a_nl', 'v262_cs_GB2', 'v178_DK', 'v275b_N2_GE03', 'v263_cs_GB1', 'c_abrv_MK', 'f85', 'c_abrv_SK', 'v243_cs_GB2', 'v281a_sq', 'c_abrv_AM', 'v281a_sk', 'v262_cs_DE2', 'c_abrv_FR', 'v281a_hr', 'c_abrv_BY', 'v263_cs_GB2', 'c_abrv_HU', 'c_abrv_AL', 'v243_cs_DE2', 'c_abrv_BA', 'v252_cs_DE2', 'v169', 'v281a_hu', 'v281a_it', 'v252_cs_DE1', 'c_abrv_RU', 'v243_cs_DE1', 'v263_cs_DE1', 'v141_11c', 'v262_cs_DE1', 'v179_DK', 'v262_cs_DE3', 'c_abrv_UA', 'v281a_hy', 'v281a_bg', 'c_abrv_BG', 'v252_cs_DE3', 'c_abrv_IT', 'v263_cs_DE3', 'v275b_N2_LT02', 'f46_IT', 'c_abrv_CH', 'v135_11c', 'f252_edulvlb_CH', 'v251b_-3', 'v281a_fr', 'v181_DK', 'v243_cs_DE3', 'v281a_ro', 'c_abrv_HR', 'v138_11c', 'c_abrv_PL', 'c_abrv_RO', 'f24_IT', 'v136_11c', 'c_abrv_LT', 'v137_11c', 'v134_11c', 'f45a', 'v140_11c', 'v227', 'v17', 'v281a_de', 'v281a_lt', 'v139_11c', 'v133_11c', 'c_abrv_RS', 'f96', 'v228b_-3', 'v96', 'f20', 'v281a_sr', 'v18', 'v45a', 'v281a_da', 'v275b_N2_GE02', 'v233b_-3', 'v232', 'c_abrv_DK', 'v256', 'v281a_ru', 'v12', 'v231b_-3', 'v13', 'v230', 'v14', 'c_abrv_CZ', 'c_abrv_AZ', 'v281a_cs', 'v281a_az', 'v19', 'v229', 'v247', 'v16', 'v260', 'v10', 'v258', 'v11', 'v20a', 'v40', 'v45b', 'v238', 'v32', 'v20b', 'v45c', 'v15', 'v96a', 'v65', 'v250', 'v29', 'v21', 'v259', 'v25', 'v88', 'v85', 'v95', 'v96b', 'v22', 'v41', 'v89', 'v94', 'v87', 'v90', 'v248a', 'v44', 'v237', 'v23', 'v27', 'v31', 'v243_r', 'v254', 'v24', 'v236', 'v91', 'v92', 'v42', 'v263_r', 'v30a', 'v30', 'v43', 'v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']

#800
# cols=['v275b_N2_AZ-SAL', 'v275b_N2_AT11', 'v275b_N2_AT34', 'v233b_NL', 'v251b_FR', 'v231b_LT', 'v231b_LK', 'v233b_CZ', 'v233b_MA', 'v275b_N2_UKL1', 'v231b_RO', 'v275b_N2_ES62', 'v275b_N2_UA43', 'v251b_SI', 'v275b_N2_ITI3', 'v275b_N2_ITH4', 'v231b_143', 'v275b_N2_UKM7', 'v251b_5', 'v275b_N2_FRF1', 'v251b_34', 'v231b_FR', 'v275b_N2_ITF6', 'v231b_DZ', 'v275b_N2_ITH3', 'v275b_N2_UKG3', 'v275b_N2_AZ-ABS', 'v275b_N2_AZ-BAL', 'v275b_N2_FRB0', 'v231b_CSHH', 'v228b_11', 'v275b_N2_NL23', 'v233b_HU', 'v233b_GE', 'v233b_BY', 'v275b_N2_UA14', 'v275b_N2_NL42', 'v275b_N2_NO02', 'v231b_LV', 'v251b_XK', 'v233b_LK', 'v275b_N2_ITG2', 'v275b_N2_FRJ1', 'v275b_N2_UA81', 'v251b_ES', 'v231b_14', 'v275b_N2_UA84', 'v275b_N2_AZ-AST', 'v231b_15', 'v233b_11', 'v231b_GE', 'v275b_N2_UKK1', 'v228b_SK', 'v275b_N2_AZ-SKR', 'v275b_N2_UA13', 'v233b_15', 'v275b_N2_NO04', 'v231b_BE', 'v231b_11', 'v275b_N2_RU81', 'v231b_US', 'v228b_14', 'v275b_N2_NL31', 'v275b_N2_ES12', 'v231b_HU', 'v275b_N2_UA51', 'v233b_MK', 'v275b_N2_AZ-GYG', 'v251b_IT', 'v231b_GR', 'v275b_N2_UKH1', 'v275b_N2_ES41', 'v251b_35', 'v231b_PT', 'v231b_AT', 'v275b_N2_PL63', 'v275b_N2_AZ-GAD', 'v251b_PL', 'v275b_N2_UKJ2', 'v275b_N2_UKJ1', 'v233b_BE', 'v275b_N2_PL84', 'v233b_HR', 'v228b_TR', 'v233b_GR', 'v275b_N2_PT18', 'v275b_N2_AZ-HAC', 'v233b_143', 'v233b_AT', 'v251b_PT', 'v275b_N2_ES42', 'v275b_N2_UA11', 'v251b_BA', 'v275b_N2_FRJ2', 'v228b_RO', 'v233b_DZ', 'v275b_N2_HU32', 'v251b_UA', 'v275b_N2_PL51', 'v228b_143', 'v275b_N2_UKD3', 'v233b_SK', 'v275b_N2_ES11', 'v275b_N2_NO06', 'v275b_N2_ES52', 'v275b_N2_UKI7', 'v275b_N2_ES70', 'v233b_ES', 'v233b_14', 'v233b_XK', 'v275b_N2_FRG0', 'v231b_ES', 'v233b_FR', 'v275b_N2_UKG2', 'v228b_FR', 'v231b_BY', 'v233b_TR', 'v275b_N2_NL11', 'v275b_N2_PL61', 'v228b_PL', 'v233b_RO', 'v275b_N2_FRL0', 'v275b_N2_UA62', 'v275b_N2_UA83', 'v275b_N2_UA53', 'v231b_MA', 'v275b_N2_PL41', 'v275b_N2_UKM9', 'v275b_N2_PL92', 'v233b_PT', 'v275b_N2_ITC3', 'v231b_HR', 'v228b_34', 'v231b_SK', 'v275b_N2_PL71', 'v275b_N2_RU51', 'v228b_UA', 'v275b_N2_PL21', 'v275b_N2_AZ-ZAR', 'v275b_N2_AZ-MAS', 'v275b_N2_FRD1', 'v275b_N2_FRH0', 'v228b_35', 'v275b_N2_UA21', 'v231b_XK', 'v275b_N2_PL22', 'v275b_N2_UA42', 'v275b_N2_HU22', 'v233b_RS', 'v275b_N2_RU71', 'v275b_N2_CZ01', 'v228b_IT', 'v275b_N2_AZ-QAZ', 'v275b_N2_RO11', 'v275b_N2_NO07', 'v275b_N2_UA63', 'v275b_N2_AZ-AGU', 'v233b_35', 'v275b_N2_BY07', 'v275b_N2_AZ-SAT', 'v231b_RS', 'v275b_N2_ES51', 'v275b_N2_UA31', 'v251b_DE', 'v275b_N2_RO12', 'v275b_N2_FRC1', 'v275b_N2_PL91', 'v275b_N2_UA41', 'v231b_TR', 'v275b_N2_HU31', 'v275b_N2_AZ-GA', 'v233b_5', 'v275b_N2_ITF3', 'v275b_N2_AZ-MI', 'v275b_N2_CZ02', 'v231b_PL', 'v275b_N2_ITH5', 'v275b_N2_RO32', 'v275b_N2_AZ-TOV', 'v275b_N2_NL21', 'v275b_N2_CH07', 'v231b_5', 'v275b_N2_BG31', 'v275b_N2_ES61', 'v275b_N2_AZ-SMX', 'v275b_N2_ITI1', 'v275b_N2_AT32', 'v251b_-1', 'v275b_N2_RO41', 'v233b_IT', 'v231b_34', 'v275b_N2_ITI4', 'v228b_BA', 'v275b_N2_UKF1', 'v275b_N2_FRE1', 'v275b_N2_HU21', 'v275b_N2_RO31', 'v275b_N2_ITC1', 'v251b_RU', 'v275b_N2_UA72', 'v275b_N2_AT21', 'v275b_N2_NO03', 'v233b_UA', 'v275b_N2_RO22', 'v275b_N2_BG32', 'v275b_N2_AT33', 'v275b_N2_CZ08', 'v275b_N2_FRK2', 'v275b_N2_FI1D', 'v275b_N2_CZ07', 'v233b_PL', 'v275b_N2_NL22', 'v275b_N2_HU23', 'v275b_N2_NO05', 'v228b_5', 'v275b_N2_FRI1', 'v275b_N2_UA71', 'v275b_N2_RU21', 'v275b_N2_RO42', 'v233b_34', 'v275b_N2_FI1B', 'v275b_N2_ITF4', 'v275b_N2_DK05', 'v231b_35', 'v275b_N2_FI19', 'v275b_N2_BY04', 'v275b_N2_HU33', 'v275b_N2_FI1C', 'v275b_N2_BY06', 'v231b_IT', 'v275b_N2_HU12', 'v275b_N2_BY02', 'v275b_N2_AM03', 'v275b_N2_BY01', 'v275b_N2_ITG1', 'v275b_N2_HU11', 'v275b_N2_NL41', 'v275b_N2_AZ-QUS', 'v275b_N2_AT31', 'v275b_N2_BY03', 'v275b_N2_PT16', 'v275b_N2_NL32', 'v275b_N2_BG42', 'v231b_DE', 'v275b_N2_CH03', 'v231b_UA', 'v275b_N2_AM04', 'v24a_IT', 'v275b_N2_DK02', 'v275b_N2_CZ06', 'v24b_IT', 'v231b_BA', 'v275b_N2_ES30', 'v275b_N2_SI04', 'v275b_N2_RO21', 'v275b_N2_RU31', 'v275b_N2_PT17', 'v275b_N2_AT22', 'v233b_BA', 'v275b_N2_RU61', 'v228b_DE', 'v275b_N2_CH06', 'v233b_DE', 'v275b_N2_ITC4', 'v275b_N2_AL01', 'v275b_N2_CZ05', 'v275b_N2_NO01', 'v275b_N2_NL33', 'v275b_N2_AZ-ISM', 'v275b_N2_AT12', 'v275b_N2_UA61', 'v221_DK', 'v275b_N2_BG34', 'v275b_N2_PT11', 'v275b_N2_RS22', 'v275b_N2_SK01', 'v275b_N2_BG33', 'v275b_N2_AZ-GOR', 'v275b_N2_AM02', 'v224_DK', 'v228b_RU', 'v223_DK', 'c_abrv_NO', 'v275b_N2_RS11', 'v275b_N2_CZ04', 'v275b_N2_CH04', 'v275b_N2_AT13', 'v183_DK', 'v275b_N2_FR10', 'v275b_N2_RU11', 'v275b_N2_SK04', 'v275b_N2_CZ03', 'v275b_N2_CH01', 'v275b_N2_SK03', 'v222_DK', 'v275b_N2_BY05', 'v275b_N2_CH05', 'v275b_N2_CH02', 'v275b_N2_RU41', 'v281a_no', 'f110', 'v275b_N2_AM01', 'v275b_N2_GE01', 'v275b_N2_HR03', 'v275b_N2_DK03', 'v275b_N2_SI03', 'v275b_N2_AZ-BA', 'v275b_N2_SK02', 'f108', 'v275b_N2_BG41', 'v275b_N2_DK04', 'v281a_pt', 'c_abrv_GB', 'c_abrv_PT', 'v275b_N2_BASRP', 'v177_DK', 'v275b_N2_LT01', 'v233b_RU', 'v275b_N2_AL02', 'v275b_N2_RS21', 'v281a_et', 'v275b_N2_DK01', 'v281a_fi', 'c_abrv_FI', 'c_abrv_SI', 'v182_DK', 'v281a_mk', 'v281a_en', 'v231b_RU', 'v275b_N2_AL03', 'v281a_lv', 'v176_DK', 'v281a_sl', 'v275b_N2_IS00', 'v275b_N2_RS12', 'v281a_uk', 'c_abrv_DE', 'c_abrv_EE', 'c_abrv_IS', 'v281a_is', 'v275b_N2_EE00', 'v252_cs_GB2', 'v275b_N2_HR04', 'v281a_bs', 'v180_DK', 'v281a_es', 'v275b_N2_-4', 'c_abrv_NL', 'v243_cs_GB1', 'v275b_N2_LV00', 'c_abrv_AT', 'v262_cs_GB1', 'c_abrv_ES', 'c_abrv_LV', 'v252_cs_GB1', 'v275b_N2_BABIH', 'v275b_N2_MK00', 'v281a_pl', 'v263_cs_DE2', 'v281a_nl', 'v262_cs_GB2', 'v178_DK', 'v275b_N2_GE03', 'v263_cs_GB1', 'c_abrv_MK', 'f85', 'c_abrv_SK', 'v243_cs_GB2', 'v281a_sq', 'c_abrv_AM', 'v281a_sk', 'v262_cs_DE2', 'c_abrv_FR', 'v281a_hr', 'c_abrv_BY', 'v263_cs_GB2', 'c_abrv_HU', 'c_abrv_AL', 'v243_cs_DE2', 'c_abrv_BA', 'v252_cs_DE2', 'v169', 'v281a_hu', 'v281a_it', 'v252_cs_DE1', 'c_abrv_RU', 'v243_cs_DE1', 'v263_cs_DE1', 'v141_11c', 'v262_cs_DE1', 'v179_DK', 'v262_cs_DE3', 'c_abrv_UA', 'v281a_hy', 'v281a_bg', 'c_abrv_BG', 'v252_cs_DE3', 'c_abrv_IT', 'v263_cs_DE3', 'v275b_N2_LT02', 'f46_IT', 'c_abrv_CH', 'v135_11c', 'f252_edulvlb_CH', 'v251b_-3', 'v281a_fr', 'v181_DK', 'v243_cs_DE3', 'v281a_ro', 'c_abrv_HR', 'v138_11c', 'c_abrv_PL', 'c_abrv_RO', 'f24_IT', 'v136_11c', 'c_abrv_LT', 'v137_11c', 'v134_11c', 'f45a', 'v140_11c', 'v227', 'v17', 'v281a_de', 'v281a_lt', 'v139_11c', 'v133_11c', 'c_abrv_RS', 'f96', 'v228b_-3', 'v96', 'f20', 'v281a_sr', 'v18', 'v45a', 'v281a_da', 'v275b_N2_GE02', 'v233b_-3', 'v232', 'c_abrv_DK', 'v256', 'v281a_ru', 'v12', 'v231b_-3', 'v13', 'v230', 'v14', 'c_abrv_CZ', 'c_abrv_AZ', 'v281a_cs', 'v281a_az', 'v19', 'v229', 'v247', 'v16', 'v260', 'v10', 'v258', 'v11', 'v20a', 'v40', 'v45b', 'v238', 'v32', 'v20b', 'v45c', 'v15', 'v96a', 'v65', 'v250', 'v29', 'v21', 'v259', 'v25', 'v88', 'v85', 'v95', 'v96b', 'v22', 'v41', 'v89', 'v94', 'v87', 'v90', 'v248a', 'v44', 'v237', 'v23', 'v27', 'v31', 'v243_r', 'v254', 'v24', 'v236', 'v91', 'v92', 'v42', 'v263_r', 'v30a', 'v30', 'v43', 'v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']

#700
cols=['v275b_N2_UKD3', 'v233b_SK', 'v275b_N2_ES11', 'v275b_N2_NO06', 'v275b_N2_ES52', 'v275b_N2_UKI7', 'v275b_N2_ES70', 'v233b_ES', 'v233b_14', 'v233b_XK', 'v275b_N2_FRG0', 'v231b_ES', 'v233b_FR', 'v275b_N2_UKG2', 'v228b_FR', 'v231b_BY', 'v233b_TR', 'v275b_N2_NL11', 'v275b_N2_PL61', 'v228b_PL', 'v233b_RO', 'v275b_N2_FRL0', 'v275b_N2_UA62', 'v275b_N2_UA83', 'v275b_N2_UA53', 'v231b_MA', 'v275b_N2_PL41', 'v275b_N2_UKM9', 'v275b_N2_PL92', 'v233b_PT', 'v275b_N2_ITC3', 'v231b_HR', 'v228b_34', 'v231b_SK', 'v275b_N2_PL71', 'v275b_N2_RU51', 'v228b_UA', 'v275b_N2_PL21', 'v275b_N2_AZ-ZAR', 'v275b_N2_AZ-MAS', 'v275b_N2_FRD1', 'v275b_N2_FRH0', 'v228b_35', 'v275b_N2_UA21', 'v231b_XK', 'v275b_N2_PL22', 'v275b_N2_UA42', 'v275b_N2_HU22', 'v233b_RS', 'v275b_N2_RU71', 'v275b_N2_CZ01', 'v228b_IT', 'v275b_N2_AZ-QAZ', 'v275b_N2_RO11', 'v275b_N2_NO07', 'v275b_N2_UA63', 'v275b_N2_AZ-AGU', 'v233b_35', 'v275b_N2_BY07', 'v275b_N2_AZ-SAT', 'v231b_RS', 'v275b_N2_ES51', 'v275b_N2_UA31', 'v251b_DE', 'v275b_N2_RO12', 'v275b_N2_FRC1', 'v275b_N2_PL91', 'v275b_N2_UA41', 'v231b_TR', 'v275b_N2_HU31', 'v275b_N2_AZ-GA', 'v233b_5', 'v275b_N2_ITF3', 'v275b_N2_AZ-MI', 'v275b_N2_CZ02', 'v231b_PL', 'v275b_N2_ITH5', 'v275b_N2_RO32', 'v275b_N2_AZ-TOV', 'v275b_N2_NL21', 'v275b_N2_CH07', 'v231b_5', 'v275b_N2_BG31', 'v275b_N2_ES61', 'v275b_N2_AZ-SMX', 'v275b_N2_ITI1', 'v275b_N2_AT32', 'v251b_-1', 'v275b_N2_RO41', 'v233b_IT', 'v231b_34', 'v275b_N2_ITI4', 'v228b_BA', 'v275b_N2_UKF1', 'v275b_N2_FRE1', 'v275b_N2_HU21', 'v275b_N2_RO31', 'v275b_N2_ITC1', 'v251b_RU', 'v275b_N2_UA72', 'v275b_N2_AT21', 'v275b_N2_NO03', 'v233b_UA', 'v275b_N2_RO22', 'v275b_N2_BG32', 'v275b_N2_AT33', 'v275b_N2_CZ08', 'v275b_N2_FRK2', 'v275b_N2_FI1D', 'v275b_N2_CZ07', 'v233b_PL', 'v275b_N2_NL22', 'v275b_N2_HU23', 'v275b_N2_NO05', 'v228b_5', 'v275b_N2_FRI1', 'v275b_N2_UA71', 'v275b_N2_RU21', 'v275b_N2_RO42', 'v233b_34', 'v275b_N2_FI1B', 'v275b_N2_ITF4', 'v275b_N2_DK05', 'v231b_35', 'v275b_N2_FI19', 'v275b_N2_BY04', 'v275b_N2_HU33', 'v275b_N2_FI1C', 'v275b_N2_BY06', 'v231b_IT', 'v275b_N2_HU12', 'v275b_N2_BY02', 'v275b_N2_AM03', 'v275b_N2_BY01', 'v275b_N2_ITG1', 'v275b_N2_HU11', 'v275b_N2_NL41', 'v275b_N2_AZ-QUS', 'v275b_N2_AT31', 'v275b_N2_BY03', 'v275b_N2_PT16', 'v275b_N2_NL32', 'v275b_N2_BG42', 'v231b_DE', 'v275b_N2_CH03', 'v231b_UA', 'v275b_N2_AM04', 'v24a_IT', 'v275b_N2_DK02', 'v275b_N2_CZ06', 'v24b_IT', 'v231b_BA', 'v275b_N2_ES30', 'v275b_N2_SI04', 'v275b_N2_RO21', 'v275b_N2_RU31', 'v275b_N2_PT17', 'v275b_N2_AT22', 'v233b_BA', 'v275b_N2_RU61', 'v228b_DE', 'v275b_N2_CH06', 'v233b_DE', 'v275b_N2_ITC4', 'v275b_N2_AL01', 'v275b_N2_CZ05', 'v275b_N2_NO01', 'v275b_N2_NL33', 'v275b_N2_AZ-ISM', 'v275b_N2_AT12', 'v275b_N2_UA61', 'v221_DK', 'v275b_N2_BG34', 'v275b_N2_PT11', 'v275b_N2_RS22', 'v275b_N2_SK01', 'v275b_N2_BG33', 'v275b_N2_AZ-GOR', 'v275b_N2_AM02', 'v224_DK', 'v228b_RU', 'v223_DK', 'c_abrv_NO', 'v275b_N2_RS11', 'v275b_N2_CZ04', 'v275b_N2_CH04', 'v275b_N2_AT13', 'v183_DK', 'v275b_N2_FR10', 'v275b_N2_RU11', 'v275b_N2_SK04', 'v275b_N2_CZ03', 'v275b_N2_CH01', 'v275b_N2_SK03', 'v222_DK', 'v275b_N2_BY05', 'v275b_N2_CH05', 'v275b_N2_CH02', 'v275b_N2_RU41', 'v281a_no', 'f110', 'v275b_N2_AM01', 'v275b_N2_GE01', 'v275b_N2_HR03', 'v275b_N2_DK03', 'v275b_N2_SI03', 'v275b_N2_AZ-BA', 'v275b_N2_SK02', 'f108', 'v275b_N2_BG41', 'v275b_N2_DK04', 'v281a_pt', 'c_abrv_GB', 'c_abrv_PT', 'v275b_N2_BASRP', 'v177_DK', 'v275b_N2_LT01', 'v233b_RU', 'v275b_N2_AL02', 'v275b_N2_RS21', 'v281a_et', 'v275b_N2_DK01', 'v281a_fi', 'c_abrv_FI', 'c_abrv_SI', 'v182_DK', 'v281a_mk', 'v281a_en', 'v231b_RU', 'v275b_N2_AL03', 'v281a_lv', 'v176_DK', 'v281a_sl', 'v275b_N2_IS00', 'v275b_N2_RS12', 'v281a_uk', 'c_abrv_DE', 'c_abrv_EE', 'c_abrv_IS', 'v281a_is', 'v275b_N2_EE00', 'v252_cs_GB2', 'v275b_N2_HR04', 'v281a_bs', 'v180_DK', 'v281a_es', 'v275b_N2_-4', 'c_abrv_NL', 'v243_cs_GB1', 'v275b_N2_LV00', 'c_abrv_AT', 'v262_cs_GB1', 'c_abrv_ES', 'c_abrv_LV', 'v252_cs_GB1', 'v275b_N2_BABIH', 'v275b_N2_MK00', 'v281a_pl', 'v263_cs_DE2', 'v281a_nl', 'v262_cs_GB2', 'v178_DK', 'v275b_N2_GE03', 'v263_cs_GB1', 'c_abrv_MK', 'f85', 'c_abrv_SK', 'v243_cs_GB2', 'v281a_sq', 'c_abrv_AM', 'v281a_sk', 'v262_cs_DE2', 'c_abrv_FR', 'v281a_hr', 'c_abrv_BY', 'v263_cs_GB2', 'c_abrv_HU', 'c_abrv_AL', 'v243_cs_DE2', 'c_abrv_BA', 'v252_cs_DE2', 'v169', 'v281a_hu', 'v281a_it', 'v252_cs_DE1', 'c_abrv_RU', 'v243_cs_DE1', 'v263_cs_DE1', 'v141_11c', 'v262_cs_DE1', 'v179_DK', 'v262_cs_DE3', 'c_abrv_UA', 'v281a_hy', 'v281a_bg', 'c_abrv_BG', 'v252_cs_DE3', 'c_abrv_IT', 'v263_cs_DE3', 'v275b_N2_LT02', 'f46_IT', 'c_abrv_CH', 'v135_11c', 'f252_edulvlb_CH', 'v251b_-3', 'v281a_fr', 'v181_DK', 'v243_cs_DE3', 'v281a_ro', 'c_abrv_HR', 'v138_11c', 'c_abrv_PL', 'c_abrv_RO', 'f24_IT', 'v136_11c', 'c_abrv_LT', 'v137_11c', 'v134_11c', 'f45a', 'v140_11c', 'v227', 'v17', 'v281a_de', 'v281a_lt', 'v139_11c', 'v133_11c', 'c_abrv_RS', 'f96', 'v228b_-3', 'v96', 'f20', 'v281a_sr', 'v18', 'v45a', 'v281a_da', 'v275b_N2_GE02', 'v233b_-3', 'v232', 'c_abrv_DK', 'v256', 'v281a_ru', 'v12', 'v231b_-3', 'v13', 'v230', 'v14', 'c_abrv_CZ', 'c_abrv_AZ', 'v281a_cs', 'v281a_az', 'v19', 'v229', 'v247', 'v16', 'v260', 'v10', 'v258', 'v11', 'v20a', 'v40', 'v45b', 'v238', 'v32', 'v20b', 'v45c', 'v15', 'v96a', 'v65', 'v250', 'v29', 'v21', 'v259', 'v25', 'v88', 'v85', 'v95', 'v96b', 'v22', 'v41', 'v89', 'v94', 'v87', 'v90', 'v248a', 'v44', 'v237', 'v23', 'v27', 'v31', 'v243_r', 'v254', 'v24', 'v236', 'v91', 'v92', 'v42', 'v263_r', 'v30a', 'v30', 'v43', 'v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']

#600
# cols=['v275b_N2_AT21', 'v275b_N2_NO03', 'v233b_UA', 'v275b_N2_RO22', 'v275b_N2_BG32', 'v275b_N2_AT33', 'v275b_N2_CZ08', 'v275b_N2_FRK2', 'v275b_N2_FI1D', 'v275b_N2_CZ07', 'v233b_PL', 'v275b_N2_NL22', 'v275b_N2_HU23', 'v275b_N2_NO05', 'v228b_5', 'v275b_N2_FRI1', 'v275b_N2_UA71', 'v275b_N2_RU21', 'v275b_N2_RO42', 'v233b_34', 'v275b_N2_FI1B', 'v275b_N2_ITF4', 'v275b_N2_DK05', 'v231b_35', 'v275b_N2_FI19', 'v275b_N2_BY04', 'v275b_N2_HU33', 'v275b_N2_FI1C', 'v275b_N2_BY06', 'v231b_IT', 'v275b_N2_HU12', 'v275b_N2_BY02', 'v275b_N2_AM03', 'v275b_N2_BY01', 'v275b_N2_ITG1', 'v275b_N2_HU11', 'v275b_N2_NL41', 'v275b_N2_AZ-QUS', 'v275b_N2_AT31', 'v275b_N2_BY03', 'v275b_N2_PT16', 'v275b_N2_NL32', 'v275b_N2_BG42', 'v231b_DE', 'v275b_N2_CH03', 'v231b_UA', 'v275b_N2_AM04', 'v24a_IT', 'v275b_N2_DK02', 'v275b_N2_CZ06', 'v24b_IT', 'v231b_BA', 'v275b_N2_ES30', 'v275b_N2_SI04', 'v275b_N2_RO21', 'v275b_N2_RU31', 'v275b_N2_PT17', 'v275b_N2_AT22', 'v233b_BA', 'v275b_N2_RU61', 'v228b_DE', 'v275b_N2_CH06', 'v233b_DE', 'v275b_N2_ITC4', 'v275b_N2_AL01', 'v275b_N2_CZ05', 'v275b_N2_NO01', 'v275b_N2_NL33', 'v275b_N2_AZ-ISM', 'v275b_N2_AT12', 'v275b_N2_UA61', 'v221_DK', 'v275b_N2_BG34', 'v275b_N2_PT11', 'v275b_N2_RS22', 'v275b_N2_SK01', 'v275b_N2_BG33', 'v275b_N2_AZ-GOR', 'v275b_N2_AM02', 'v224_DK', 'v228b_RU', 'v223_DK', 'c_abrv_NO', 'v275b_N2_RS11', 'v275b_N2_CZ04', 'v275b_N2_CH04', 'v275b_N2_AT13', 'v183_DK', 'v275b_N2_FR10', 'v275b_N2_RU11', 'v275b_N2_SK04', 'v275b_N2_CZ03', 'v275b_N2_CH01', 'v275b_N2_SK03', 'v222_DK', 'v275b_N2_BY05', 'v275b_N2_CH05', 'v275b_N2_CH02', 'v275b_N2_RU41', 'v281a_no', 'f110', 'v275b_N2_AM01', 'v275b_N2_GE01', 'v275b_N2_HR03', 'v275b_N2_DK03', 'v275b_N2_SI03', 'v275b_N2_AZ-BA', 'v275b_N2_SK02', 'f108', 'v275b_N2_BG41', 'v275b_N2_DK04', 'v281a_pt', 'c_abrv_GB', 'c_abrv_PT', 'v275b_N2_BASRP', 'v177_DK', 'v275b_N2_LT01', 'v233b_RU', 'v275b_N2_AL02', 'v275b_N2_RS21', 'v281a_et', 'v275b_N2_DK01', 'v281a_fi', 'c_abrv_FI', 'c_abrv_SI', 'v182_DK', 'v281a_mk', 'v281a_en', 'v231b_RU', 'v275b_N2_AL03', 'v281a_lv', 'v176_DK', 'v281a_sl', 'v275b_N2_IS00', 'v275b_N2_RS12', 'v281a_uk', 'c_abrv_DE', 'c_abrv_EE', 'c_abrv_IS', 'v281a_is', 'v275b_N2_EE00', 'v252_cs_GB2', 'v275b_N2_HR04', 'v281a_bs', 'v180_DK', 'v281a_es', 'v275b_N2_-4', 'c_abrv_NL', 'v243_cs_GB1', 'v275b_N2_LV00', 'c_abrv_AT', 'v262_cs_GB1', 'c_abrv_ES', 'c_abrv_LV', 'v252_cs_GB1', 'v275b_N2_BABIH', 'v275b_N2_MK00', 'v281a_pl', 'v263_cs_DE2', 'v281a_nl', 'v262_cs_GB2', 'v178_DK', 'v275b_N2_GE03', 'v263_cs_GB1', 'c_abrv_MK', 'f85', 'c_abrv_SK', 'v243_cs_GB2', 'v281a_sq', 'c_abrv_AM', 'v281a_sk', 'v262_cs_DE2', 'c_abrv_FR', 'v281a_hr', 'c_abrv_BY', 'v263_cs_GB2', 'c_abrv_HU', 'c_abrv_AL', 'v243_cs_DE2', 'c_abrv_BA', 'v252_cs_DE2', 'v169', 'v281a_hu', 'v281a_it', 'v252_cs_DE1', 'c_abrv_RU', 'v243_cs_DE1', 'v263_cs_DE1', 'v141_11c', 'v262_cs_DE1', 'v179_DK', 'v262_cs_DE3', 'c_abrv_UA', 'v281a_hy', 'v281a_bg', 'c_abrv_BG', 'v252_cs_DE3', 'c_abrv_IT', 'v263_cs_DE3', 'v275b_N2_LT02', 'f46_IT', 'c_abrv_CH', 'v135_11c', 'f252_edulvlb_CH', 'v251b_-3', 'v281a_fr', 'v181_DK', 'v243_cs_DE3', 'v281a_ro', 'c_abrv_HR', 'v138_11c', 'c_abrv_PL', 'c_abrv_RO', 'f24_IT', 'v136_11c', 'c_abrv_LT', 'v137_11c', 'v134_11c', 'f45a', 'v140_11c', 'v227', 'v17', 'v281a_de', 'v281a_lt', 'v139_11c', 'v133_11c', 'c_abrv_RS', 'f96', 'v228b_-3', 'v96', 'f20', 'v281a_sr', 'v18', 'v45a', 'v281a_da', 'v275b_N2_GE02', 'v233b_-3', 'v232', 'c_abrv_DK', 'v256', 'v281a_ru', 'v12', 'v231b_-3', 'v13', 'v230', 'v14', 'c_abrv_CZ', 'c_abrv_AZ', 'v281a_cs', 'v281a_az', 'v19', 'v229', 'v247', 'v16', 'v260', 'v10', 'v258', 'v11', 'v20a', 'v40', 'v45b', 'v238', 'v32', 'v20b', 'v45c', 'v15', 'v96a', 'v65', 'v250', 'v29', 'v21', 'v259', 'v25', 'v88', 'v85', 'v95', 'v96b', 'v22', 'v41', 'v89', 'v94', 'v87', 'v90', 'v248a', 'v44', 'v237', 'v23', 'v27', 'v31', 'v243_r', 'v254', 'v24', 'v236', 'v91', 'v92', 'v42', 'v263_r', 'v30a', 'v30', 'v43', 'v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']

#500
# cols =  ['f110', 'v275b_N2_AM01', 'v275b_N2_GE01', 'v275b_N2_HR03', 'v275b_N2_DK03', 'v275b_N2_SI03', 'v275b_N2_AZ-BA', 'v275b_N2_SK02', 'f108', 'v275b_N2_BG41', 'v275b_N2_DK04', 'v281a_pt', 'c_abrv_GB', 'c_abrv_PT', 'v275b_N2_BASRP', 'v177_DK', 'v275b_N2_LT01', 'v233b_RU', 'v275b_N2_AL02', 'v275b_N2_RS21', 'v281a_et', 'v275b_N2_DK01', 'v281a_fi', 'c_abrv_FI', 'c_abrv_SI', 'v182_DK', 'v281a_mk', 'v281a_en', 'v231b_RU', 'v275b_N2_AL03', 'v281a_lv', 'v176_DK', 'v281a_sl', 'v275b_N2_IS00', 'v275b_N2_RS12', 'v281a_uk', 'c_abrv_DE', 'c_abrv_EE', 'c_abrv_IS', 'v281a_is', 'v275b_N2_EE00', 'v252_cs_GB2', 'v275b_N2_HR04', 'v281a_bs', 'v180_DK', 'v281a_es', 'v275b_N2_-4', 'c_abrv_NL', 'v243_cs_GB1', 'v275b_N2_LV00', 'c_abrv_AT', 'v262_cs_GB1', 'c_abrv_ES', 'c_abrv_LV', 'v252_cs_GB1', 'v275b_N2_BABIH', 'v275b_N2_MK00', 'v281a_pl', 'v263_cs_DE2', 'v281a_nl', 'v262_cs_GB2', 'v178_DK', 'v275b_N2_GE03', 'v263_cs_GB1', 'c_abrv_MK', 'f85', 'c_abrv_SK', 'v243_cs_GB2', 'v281a_sq', 'c_abrv_AM', 'v281a_sk', 'v262_cs_DE2', 'c_abrv_FR', 'v281a_hr', 'c_abrv_BY', 'v263_cs_GB2', 'c_abrv_HU', 'c_abrv_AL', 'v243_cs_DE2', 'c_abrv_BA', 'v252_cs_DE2', 'v169', 'v281a_hu', 'v281a_it', 'v252_cs_DE1', 'c_abrv_RU', 'v243_cs_DE1', 'v263_cs_DE1', 'v141_11c', 'v262_cs_DE1', 'v179_DK', 'v262_cs_DE3', 'c_abrv_UA', 'v281a_hy', 'v281a_bg', 'c_abrv_BG', 'v252_cs_DE3', 'c_abrv_IT', 'v263_cs_DE3', 'v275b_N2_LT02', 'f46_IT', 'c_abrv_CH', 'v135_11c', 'f252_edulvlb_CH', 'v251b_-3', 'v281a_fr', 'v181_DK', 'v243_cs_DE3', 'v281a_ro', 'c_abrv_HR', 'v138_11c', 'c_abrv_PL', 'c_abrv_RO', 'f24_IT', 'v136_11c', 'c_abrv_LT', 'v137_11c', 'v134_11c', 'f45a', 'v140_11c', 'v227', 'v17', 'v281a_de', 'v281a_lt', 'v139_11c', 'v133_11c', 'c_abrv_RS', 'f96', 'v228b_-3', 'v96', 'f20', 'v281a_sr', 'v18', 'v45a', 'v281a_da', 'v275b_N2_GE02', 'v233b_-3', 'v232', 'c_abrv_DK', 'v256', 'v281a_ru', 'v12', 'v231b_-3', 'v13', 'v230', 'v14', 'c_abrv_CZ', 'c_abrv_AZ', 'v281a_cs', 'v281a_az', 'v19', 'v229', 'v247', 'v16', 'v260', 'v10', 'v258', 'v11', 'v20a', 'v40', 'v45b', 'v238', 'v32', 'v20b', 'v45c', 'v15', 'v96a', 'v65', 'v250', 'v29', 'v21', 'v259', 'v25', 'v88', 'v85', 'v95', 'v96b', 'v22', 'v41', 'v89', 'v94', 'v87', 'v90', 'v248a', 'v44', 'v237', 'v23', 'v27', 'v31', 'v243_r', 'v254', 'v24', 'v236', 'v91', 'v92', 'v42', 'v263_r', 'v30a', 'v30', 'v43', 'v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']

#300
# cols = ['v225', 'v257', 'v222', 'v45', 'v2', 'v262_r', 'v224', 'age_r2', 'v20', 'f30a', 'v252_r', 'v264', 'v190', 'v101', 'v192', 'v243_ISCED97', 'v30c', 'v30b', 'v248', 'v252_ISCED97', 'v193', 'v112', 'v67', 'v34', 'v172', 'v152', 'c_abrv_GE', 'v204', 'v28', 'v223', 'v111_4', 'v86', 'v61', 'v265', 'v148', 'v245', 'v171', 'v70', 'v26', 'v99', 'v66', 'v281a_ka', 'v268', 'v162', 'v77', 'v234', 'v239b', 'v7', 'v71', 'v166', 'v239a', 'v164', 'v9', 'v100', 'v252_ISCED_1', 'v221', 'v252_EISCED', 'v68', 'v263_ISCED97', 'v252_edulvlb_1', 'v33', 'v252_8cat', 'v243_edulvlb_1', 'v98', 'v262_ISCED97', 'v35', 'v165', 'v113', 'v117', 'v243_ISCED_1', 'v170', 'v122', 'v253', 'v108', 'v130', 'v252_ISCED_2b', 'v114', 'v120', 'v197', 'v126', 'v118', 'v207', 'v263_ISCED_1', 'v123', 'v261_r', 'v243_8cat', 'v8', 'v252_ISCED_2', 'year', 'v127', 'v262_edulvlb_1', 'v263_edulvlb_1', 'v243_EISCED', 'v121', 'v167', 'v252_edulvlb_2', 'v263_EISCED', 'v206', 'v280', 'v78', 'v189', 'v128', 'v129', 'v183', 'v249', 'v272', 'v110', 'v262_ISCED_1', 'v119', 'v131', 'v76', 'v195', 'v132', 'v1', 'v262_EISCED', 'v116', 'v46', 'v243_ISCED_2b', 'v194', 'v168', 'v177', 'v270', 'v263_8cat', 'v74', 'v255_egp', 'v124', 'v205', 'v267', 'v146', 'v141', 'v273', 'v150', 'v252_ISCED_3', 'v255_ESeC', 'v214', 'v36', 'v125', 'v37', 'v252_edulvlb', 'v263_ISCED_2b', 'v217', 'v212', 'v4', 'v181', 'v274', 'v184', 'v180', 'v149', 'v191', 'v243_ISCED_2', 'v111', 'v80', 'v49', 'v210', 'v239_r', 'v213', 'v211', 'v173', 'v178', 'v262_8cat', 'v271', 'v176', 'v97', 'v200', 'v220', 'v179', 'v215', 'v73', 'v269', 'v276_r', 'v109', 'v244', 'v48', 'v142', 'v262_ISCED_2b', 'v72', 'v135', 'v201', 'age_r', 'v263_ISCED_2', 'v263_edulvlb_2', 'v240', 'v182', 'v202', 'v262_ISCED_2', 'v209', 'v203', 'v145', 'v3', 'v208', 'v79', 'v243_edulvlb_2', 'v219', 'v216', 'v199', 'v262_edulvlb_2', 'v263_ISCED_3', 'v262_ISCED_3', 'v75', 'v47', 'v243_ISCED_3', 'v147', 'v50', 'v242_r', 'age_r3', 'v159', 'v69', 'v262_edulvlb', 'v263_edulvlb', 'v243_edulvlb', 'v255_ISCO_2', 'v83', 'v255_ISEI', 'v255_SIOPS', 'v138', 'v84', 'v246_ESeC', 'v218', 'v235', 'v81', 'v163', 'v246_egp', 'v175_LR', 'v39', 'v136', 'v157', 'v252_cs', 'v38', 'v266', 'v137', 'v151', 'fw_start', 'v105', 'v139', 'v174_LR', 'v187', 'fw_end', 'v186', 'v133', 'v278a', 'v198', 'v261', 'v143', 'v107', 'v140', 'v144', 'v279a', 'v104', 'v102', 'v188', 'v185', 'v82', 'v242', 'v103', 'v106', 'v161', 'v175_cs', 'v246_ISCO_2', 'v241', 'v246_SIOPS', 'v246_ISEI', 'v160', 'v134', 'v263_cs', 'v174_cs', 'v279b', 'v278b', 'v262_cs', 'v153', 'v155', 'age', 'v226', 'v278c_r', 'v279d_r', 'v58', 'v279c_r', 'v243_cs', 'v282', 'v158', 'v261_ppp', 'v196', 'v277', 'v55', 'v59', 'v156', 'v93', 'v5', 'v60', 'v53', 'v154', 'v62', 'v51', 'v52_cs', 'v115', 'v52', 'v57', 'v54', 'v56', 'v64', 'v63']


X_train = dataset.getTrain().loc[:,cols].values
X_test = dataset.getTest().loc[:,cols].values

y_train_data = read_csv('../y_train.csv')
y_train = y_train_data.values[:,1]
for i in range(len(y_train)):
    if y_train[i] == -1:
        y_train[i] = 0

KeyboardInterrupt: 

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.20, random_state=441)

In [ ]:
xgb_model = xgb.XGBClassifier(objective="multi:softmax").fit(X_train, y_train) 

In [ ]:
# Using default num_leaves and learning_rate:
    # -0.846532 using {'feature_fraction': 0.26, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 175, 'num_leaves': 31, 'subsample': 0.5}
lgbm_model = lgb.LGBMClassifier(objective='multiclass').fit(X_train, y_train)

In [ ]:
# Generate predicted probabilities from both models
xgb_probs = xgb_model.predict_proba(X_val)
lgb_probs = lgbm_model.predict_proba(X_val)

In [ ]:
stacked = np.concatenate((xgb_probs, lgb_probs), axis=1)

In [ ]:
#Test logistic meta-model
lmodel = LogisticRegressionCV(random_state=441, cv=5, max_iter=1000, scoring="neg_log_loss")
lmodel.fit(stacked,y_val)

In [ ]:
print(lmodel.score(stacked, y_val))

In [ ]:
# #Test xgb meta-model
# stack_model = xgb.XGBClassifier(
#                 objective="multi:softmax", 
#                 random_state=441, 
#                 # max_depth=8, 
#                 # max_leaves=66, 
#                 # learning_rate=0.1, 
#                 # subsample=0.8, 
#                 # min_child_weight = 8,
#                 # colsample_bytree = 0.6,
#                 # gamma = 0.1
# )
# #0.2 train/val
# #Best: -0.849004 using {'colsample_bytree': 0.7, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.8}
# kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=441)


# # all
# #Best: -0.848961 using {'colsample_bytree': 0.7, 'gamma': 0.2, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}

# #900
# #Best: -0.847744 using {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}

# #800
# #Best: -0.846638 using {'colsample_bytree': 0.7, 'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}

# #700 !!!!
# #Best: -0.846632 using {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.12, 'max_depth': 2, 'max_leaves': 2, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}

# #600
# #Best: -0.846709 using {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}


# #500
# #Best: -0.849291 using {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 100, 'subsample': 0.8}

# #300
# #Best: -0.850479 using {'colsample_bytree': 0.7, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 2, 'max_leaves': 3, 'min_child_weight': 4, 'n_estimators': 105, 'subsample': 0.8}

# # param_grid = {
# #     'max_depth': [2,3],
# #     'max_leaves': [2,3],
# #     'learning_rate': [0.1, 0.12],
# #     'min_child_weight': [4],
# #     'subsample': [0.8],
# #     'colsample_bytree': [0.7, 0.8],
# #     'gamma': [0, 0.1, 0.2],
# #     'n_estimators': [100, 105]
# # }

# # param_grid = {
# #     'colsample_bytree': [0.7], 
# #     'gamma': [0], 
# #     'learning_rate': [0.13], 
# #     'max_depth': [2], 
# #     'max_leaves': [2], 
# #     'min_child_weight': [4], 
# #     'subsample': [0.8],
# #     'n_estimators': [100]
# # }

# grid_search = GridSearchCV(stack_model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
# grid_result = grid_search.fit(stacked, y_val)

# print("\nBest: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

# # print results
# means = grid_result.cv_results_['mean_test_score']
# params = grid_result.cv_results_['params']
# for mean, param in zip(means, params):
#  print("%f with: %r" % (mean, param))



In [ ]:
stacked_model = xgb.XGBClassifier(
                objective="multi:softmax", 
                random_state=441, 
                max_depth=2, 
                max_leaves=2, 
                learning_rate=0.12, 
                subsample=0.8, 
                min_child_weight = 4,
                colsample_bytree = 0.7,
                gamma = 0,
                n_estimators = 105
).fit(stacked, y_val)

In [ ]:
xgb_test = xgb_model.predict_proba(X_test)
lgb_test = lgbm_model.predict_proba(X_test)
stacked_test = np.concatenate((xgb_test, lgb_test), axis=1)

In [ ]:
prob = stacked_model.predict_proba(stacked_test)

In [ ]:
test = np.c_[ np.ones(11438), prob ]              # add a column

for i in range(len(test)):
    test[i][0] = i

np.savetxt('stack_default.csv', test, delimiter=',', header="id,no answer,very important,quite important,not important,not at all important", comments="", fmt='%1.4f') 